# Chroma Vector Database
- Chroma는 대규모 언어 모델(LLM) 애플리케이션 구축을 위해 설계된 AI 네이티브 **오픈 소스 벡터 데이터베이스**다.    
- 임베딩 저장소, 쿼리 및 검색 등의 핵심 기능을 제공하여 개발자들이 효율적으로 작업할 수 있도록 돕는다. 
- https://www.trychroma.com/
  
## Chroma의 주요 특징

- **오픈 소스 라이선스** 
  - Apache 2.0 라이선스에 따라 제공되어 누구나 자유롭게 사용하고 수정할 수 있다. 
- **다양한 개발 환경 지원**
  -  Python 및 JavaScript/TypeScript SDK를 지원하여 다양한 Langchain 과 연동하여 활용할 수 있다. 
- **유연한 데이터 저장 옵션**
  -  HTTP 방식, 디스크 저장 방식, 인메모리 방식을 선택하여 데이터를 저장할 수 있어 사용자 입장에서 매우 편리하다. 
- **간편한 사용법** 
  - 설치 및 사용법이 매우 간단하여 빠르게 프로토타입을 개발하고 검증할 수 있다. 

## 설치
- pip로 chromadb 설치시 **windows**에서는 c컴파일러 관련되어 에러가 난다. **conda 를 이용해 설치한다.**
- `conda install conda-forge::chromadb`
- `pip install langchain-chroma`
- `pip install chromadb`

# Chroma API 를 이용해 연동
- https://docs.trychroma.com/

In [1]:
import chromadb

In [10]:
from uuid import uuid4
uuid4()

UUID('aea44c69-754c-4530-8e5f-b29192d890c2')

In [3]:
from uuid import uuid4

# 추가할 데이터
document_list = [
        "This is a document about pineapple",
        "This is a document about oranges",
        "This is a document about sports",
        "This is a document about langchain",
]
ids = [str(uuid4()) for _ in range(len(document_list))]                         # DB 저장 시 지정할 각 문서들의 ID 생성

In [8]:
ids

['01c0b536-6a91-4a72-8e7a-a1ed020312d5',
 '402ddb82-87ec-491f-b247-9eaf96881043',
 '94043613-c8e0-45af-b77e-8e0ed4fade71',
 '22226f66-6e05-44d9-9cb1-500af61259a9']

In [13]:
# 외부 Embedding 모델

from dotenv import load_dotenv
import chromadb.utils.embedding_functions as embedding_functions
import os
print(load_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=OPENAI_API_KEY,
                model_name="text-embedding-3-small"
            )


True


In [14]:
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [15]:
# collection - Database
# Chroma DB와 연결
# import chromadb

# # client = chromadb.Client()                                                      # InMemory DB (데이터를 메모리에 저장)
# client = chromadb.PersistentClient(path="vector_store/chroma/my_db")            # Local 파일에 저장
# # client = chromadb.HttpClient(host="ip주소", port=8000)                           # 서버로 서비스하는 chromadb에 연결

# # collections(DB)를 생성
# collection_name = "test_db"
# collection = client.create_collections(
#     name=collection_name,
#     get_or_create=True                                                          # collections가 있으면 연결, 없으면 생성
#     # False: 이미 있는 collection 이면 Exception
# )

import chromadb

# PersistentClient로 로컬 파일에 저장
# client = chromadb.PersistentClient(path="vector_store/chroma/my_db")
client = chromadb.Client()
# collection(DB)을 생성 또는 연결
collection_name = "test_db"
collection = client.get_or_create_collection(
    name=collection_name,
    metadata={"hnsw:space":"cosine"},
    embedding_function = openai_ef
    )


In [16]:
###### 데이터 추가
collection.add(documents = document_list, ids = ids)

In [17]:
###### 유사도 검색
result = collection.query(
    query_texts = ["deeplearning", "hawaii"],               # 질문
    n_results=2,                                            # 검색 결과 수
)
result

{'ids': [['330c62f3-aaca-4d07-adcc-006be4dbeeff',
   'a57f4eb5-60cc-453d-8a61-54c1fa41437d'],
  ['a57f4eb5-60cc-453d-8a61-54c1fa41437d',
   '330c62f3-aaca-4d07-adcc-006be4dbeeff']],
 'embeddings': None,
 'documents': [['This is a document about langchain',
   'This is a document about pineapple'],
  ['This is a document about pineapple',
   'This is a document about langchain']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None], [None, None]],
 'distances': [[0.7781242728233337, 0.814582347869873],
  [0.7556754350662231, 0.8807660341262817]]}

# Langchain을 이용해 Chroma 연동

## Data 준비

In [21]:
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
    id=3,
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
    id=4,
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
    id=5,
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
    id=6,
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
    id=7,
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
    id=8,
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
    id=9,
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
    id=10,
)
document_list = [document_1,document_2,document_3,document_4,document_5,document_6,document_7,document_8,document_9,document_10]
ids = [str(uuid4()) for _ in range(len(document_list))]

## Vector Store 생성, 연결
- Chroma.from_documents()
  - VectorStore를 초기화(생성)하고 문서를 추가한다.
  - persist_directory를 지정하지 않으면 메모리에 저장된다.
- Chroma()
  - VectorStore와 연결.

In [22]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

COLLECTION_NAME = "example"                                         # 컬렉션 이름(RDB의 Database 개념)
PERSISTENT_PATH = "vector_store/chroma/example_db"                  # 저장할 로컬 경로

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 1. 연결(생성) 하면서 document들을 저장(upsert) -> RDB에서는 upsert가 insert
vector_store = Chroma.from_documents(
    documents=document_list,
    ids=ids,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSISTENT_PATH
)

In [ ]:
# 2. 연결(생성)
vector_store2 = Chroma(
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSISTENT_PATH
)

## VectorStore 정보 확인

In [25]:
coll = vector_store._collection                        # 연결된 collection 정보 확인


In [26]:
coll.count()

10

## Add (추가)

In [27]:
document_11 = Document(
    page_content="랭체인은 대규모 언어 모델(LLM)을 효과적으로 활용하기 위한 도구와 프레임워크를 제공하는 오픈소스 라이브러리입니다.",
    metadata={"source": "tweet"},
    id=10,
)

document_12 = Document(
    page_content="랭체인은 체인 구조를 사용하여 여러 LLM 작업을 연결하고, 이를 통해 더 복잡하고 맞춤화된 자연어 처리 애플리케이션을 개발할 수 있게 합니다",
    metadata={"source": "tweet"},
    id=10,
)

document_13 = Document(
    page_content="랭체인, AI 활용의 새 시대를 열다: 복잡한 언어 처리도 간단하게!",
    metadata={"source": "news"},
    id=10,
)

In [28]:
vector_store.add_documents(
    [document_11, document_12, document_13],
    ids = [str(uuid4()), str(uuid4()), str(uuid4())]
)

['debffbaa-ec69-4adf-b5b6-dcbcf2cafca2',
 '28e651c9-3e19-4fa6-b4a5-33e2aa6ce9f1',
 'f1739290-7528-4be6-8f79-2ae101979d19']

In [29]:
coll.count()

13

## Update(갱신)

In [34]:
new_document_13 = Document(
    page_content="랭체인, AI 활용의 새 시대를 열다: 복잡한 언어 처리도 간단하게?",
    metadata={"source": "news"},
    id=10,
)

In [35]:
vector_store.update_document(
    document_id="f1739290-7528-4be6-8f79-2ae101979d19",             # 바꿀 문서의 ID
    document=new_document_13                                        # 바꿀 내용을 가진 Document 객체
)

In [37]:
update_document_12 = Document(
    page_content="랭체인은 체인 구조를 사용하여 여러 LLM 작업을 연결한다.",
    metadata={"source": "website"},
)

update_document_13 = Document(
    page_content="랭체인, AI 활용의 새 시대를 열다: 복잡한 언어 처리도 간단하게!",
    metadata={"source": "news"},
)
update_docs = [update_document_12, update_document_13]
update_ids = ['28e651c9-3e19-4fa6-b4a5-33e2aa6ce9f1',
  'f1739290-7528-4be6-8f79-2ae101979d19']

vector_store.update_documents(documents=update_docs, ids=update_ids)

In [38]:
# 전체 데이터 조회
# coll.get()
vector_store.get()

{'ids': ['5630d14d-45d1-4339-a6cf-10abc6c311a5',
  'b885ef90-1f90-4f81-9d88-52b5a3c1dc24',
  '2d4c0f7d-e700-41bb-a1f1-a71ec3233c04',
  '7d6902f2-05b1-4357-9b8e-919538dafa9b',
  'a2b3d8b2-f4cd-4db4-beca-db2903c581e8',
  'd1236fd3-a0d9-4abc-b66b-986a5f0a767e',
  'b8dad090-16d7-4616-ad13-cd1e7252e7be',
  '82172976-e1e3-447e-8453-482cfe87bdd2',
  'e063b2e2-e467-4d72-9025-3f8f1ffb9ff6',
  '4286fffe-c892-402c-b68d-7652b982962e',
  'debffbaa-ec69-4adf-b5b6-dcbcf2cafca2',
  '28e651c9-3e19-4fa6-b4a5-33e2aa6ce9f1',
  'f1739290-7528-4be6-8f79-2ae101979d19'],
 'embeddings': None,
 'documents': ['I had chocolate chip pancakes and scrambled eggs for breakfast this morning.',
  'The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.',
  'Building an exciting new project with LangChain - come check it out!',
  'Robbers broke into the city bank and stole $1 million in cash.',
  "Wow! That was an amazing movie. I can't wait to see it again.",
  'Is the new iPhone worth the 

## Delete(삭제)

In [39]:
del_ids = ['b885ef90-1f90-4f81-9d88-52b5a3c1dc24',
  '2d4c0f7d-e700-41bb-a1f1-a71ec3233c04']
vector_store.delete(ids=del_ids)                               # [삭제할 문서들의 id]


In [40]:
coll.count()

11

## Query(조회)
- `similarity_search(query, k, filter)`
  - 저장되 있는 item들 중 질의와 가장 유사한 것 k개를 찾는다. 
  - 찾은 결과를 filter 조건으로 필터링 한다. filter 조건은 meta-data의 정보를 이용한다.
  - 질의어(query)는 text(자연어)로 입력한다.
- `similarity_search_with_score(query, k, filter)`
  - 저장되 있는 item들 중 질의와 가장 유사한 것 k개를 찾아 유사도 점수와 함께 반환
- `similarity_search_by_vector(embedding, k, filter)`
  - Embedding Vector 를 질의로 입력한다. (질의(query)를 문장이 아니라 embedding vector로 입력.) 

In [41]:
results = vector_store.similarity_search(
    query="Langchain이란 무엇인가요?",
    k=3
)
results

[Document(id='debffbaa-ec69-4adf-b5b6-dcbcf2cafca2', metadata={'source': 'tweet'}, page_content='랭체인은 대규모 언어 모델(LLM)을 효과적으로 활용하기 위한 도구와 프레임워크를 제공하는 오픈소스 라이브러리입니다.'),
 Document(id='28e651c9-3e19-4fa6-b4a5-33e2aa6ce9f1', metadata={'source': 'website'}, page_content='랭체인은 체인 구조를 사용하여 여러 LLM 작업을 연결한다.'),
 Document(id='82172976-e1e3-447e-8453-482cfe87bdd2', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [46]:
results = vector_store.similarity_search_with_score(
    query="다이어트에 좋은 음식은 뭐가 있을까요?",
    k=3,
    # filter={"source":"tweet"}                               # metadata의 source키값이 tweet
    filter={"source":{"$ne":"news"}}                         # news가 아닌 값
)
# 1. filter에 설정과 metadata를 비교해서 조회
# 2. 1에서 조회된 문서들과 query간의 유사도를 체크
results

[(Document(id='5630d14d-45d1-4339-a6cf-10abc6c311a5', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
  1.6316311359405518),
 (Document(id='4286fffe-c892-402c-b68d-7652b982962e', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
  1.8168842792510986),
 (Document(id='d1236fd3-a0d9-4abc-b66b-986a5f0a767e', metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.'),
  1.8391940593719482)]